In [1]:
# -*- coding: utf-8 -*-
'''
TopQuant-简称TQ极宽智能量化回溯分析系统，培训课件-配套教学python课件程序

Top极宽量化(原zw量化)，Python量化第一品牌 
by Top极宽·量化开源团队 2017.10.1 首发

网站： www.TopQuant.vip      www.ziwang.com
QQ群: Top极宽量化1群，124134140
      Top极宽量化2群，650924099
      Top极宽量化3群，450853713
  
'''
#1
import os,arrow
import pandas as pd
import numpy as np
#2
import keras
from keras import initializers,models,layers
from keras.models import Sequential
from keras.layers import Flatten,Dense, Input, Dropout, Embedding,SimpleRNN,Bidirectional,LSTM,Conv1D, GlobalMaxPooling1D,Activation,MaxPooling1D,GlobalAveragePooling1D
from keras.optimizers import RMSprop
from keras.utils import plot_model

#3
import tensorlayer as tl
import tensorflow as tf

#4
import zsys
import ztools as zt
import ztools_str as zstr
import ztools_data as zdat
import ztools_draw as zdr
import ztools_tq as ztq
import zpd_talib as zta
#
import zai_keras as zks

#
#------------------------------------

#1
print('\n#1,set.sys')
pd.set_option('display.width', 450)    
pd.set_option('display.float_format', zt.xfloat3)    
rlog='ailib/log_tmp'
if os.path.exists(rlog):tf.gfile.DeleteRecursively(rlog)

#2.1 LSTM模型输入数据模型是三维
print('\n#2.1,读取数据')
rss,fsgn,ksgn='ailib/TDS/','TDS2_sz50','avg'
xlst=zsys.TDS_xlst9
zt.prx('xlst',xlst)
num_in,num_out=len(xlst),1
print('\nnum_in,num_out:',num_in,num_out)

#
df_train,df_test,x_train,y_train,x_test, y_test=zdat.frd_TDS(rss,fsgn,ksgn,xlst)
print('\ndf_test.tail()')
print(df_test.tail())
print('\nx_train.shape,',x_train.shape)
print('\ntype(x_train),',type(x_train))

#
#2.2
print('\n#2.2,转换数据格式shape')
rxn,txn=x_train.shape[0],x_test.shape[0]
x_train,x_test = x_train.reshape(rxn,num_in,-1),x_test.reshape(txn,num_in,-1)
print('\nx_train.shape,',x_train.shape)
print('\ntype(x_train),',type(x_train))



#3
print('\n#3,model建立神经网络模型')
mx=zks.lstm010(num_in,num_out)
#
mx.summary()
plot_model(mx, to_file='tmp/lstm010.png')


#4 模型训练
print('\n#4 模型训练 fit')
tbCallBack = keras.callbacks.TensorBoard(log_dir=rlog,write_graph=True, write_images=True)
tn0=arrow.now()
mx.fit(x_train, y_train, epochs=50, batch_size=512,callbacks=[tbCallBack])
tn=zt.timNSec('',tn0,True)
mx.save('tmp/lstm010.dat')

#5 利用模型进行预测 predict
print('\n#5 模型预测 predict')
tn0=arrow.now()
y_pred = mx.predict(x_test)
tn=zt.timNSec('',tn0,True)
df_test['y_pred']=zdat.ds4x(y_pred,df_test.index,True)
df_test.to_csv('tmp/df_lstm010.csv',index=False)


#6
print('\n#6 acc准确度分析')
print('\nky0=10')
df=df_test
dacc,dfx,a10=ztq.ai_acc_xed2ext(df.y,df.y_pred,ky0=10,fgDebug=True)


Using TensorFlow backend.



#1,set.sys

#2.1,读取数据

 xlst
['open', 'high', 'low', 'close', 'volume', 'avg', 'ma_2', 'ma_3', 'ma_5', 'ma_10', 'ma_15', 'ma_20', 'ma_25', 'ma_30', 'ma_50', 'ma_100', 'xyear', 'xmonth', 'xday', 'xday_week', 'xday_year', 'xweek_year']

num_in,num_out: 22 1

df_test.tail()
      open  high   low  close      volume   avg  ma_2  ma_3  ma_5  ma_10      ...       xyear  xmonth  xday  xday_week  xday_year  xweek_year     y  price  price_next  price_change
3065 3.280 3.290 3.270  3.290  858158.000 3.280 3.280 3.280 3.280  3.310      ...        1970       1     1          3          1           0 3.310  3.280       3.310       100.915
3066 3.310 3.310 3.270  3.290 1654384.000 3.290 3.290 3.280 3.280  3.310      ...        1970       1     1          3          1           0 3.310  3.290       3.310       100.608
3067 3.320 3.320 3.300  3.310 1210668.000 3.310 3.300 3.290 3.290  3.300      ...        1970       1     1          3          1           0 3.310  3.310       3.310       100.000
306

In [5]:
# numpy 中reshape函数的三种常见相关用法 https://blog.csdn.net/qq_29831163/article/details/90112000

arr=np.arange(16).reshape(2,8)
print(arr)

[[ 0  1  2  3  4  5  6  7]
 [ 8  9 10 11 12 13 14 15]]


In [6]:
arr.shape

(2, 8)

In [8]:
arr.reshape(4,-1)

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15]])

In [9]:
arr.reshape(8,-1)

array([[ 0,  1],
       [ 2,  3],
       [ 4,  5],
       [ 6,  7],
       [ 8,  9],
       [10, 11],
       [12, 13],
       [14, 15]])

In [10]:
arr.reshape(-1,8)

array([[ 0,  1,  2,  3,  4,  5,  6,  7],
       [ 8,  9, 10, 11, 12, 13, 14, 15]])

In [11]:

np.arange(1,13,2)

array([ 1,  3,  5,  7,  9, 11])

In [3]:
market_data=pd.read_csv('/Users/bluegrey/Downloads/market_train_full.csv')
print(market_data.shape)
# (4072956, 17)

market_data.head(7)

(4072956, 17)


,Unnamed: 0,time,assetCode,assetName,volume,close,open,returnsClosePrevRaw1,returnsOpenPrevRaw1,returnsClosePrevMktres1,returnsOpenPrevMktres1,returnsClosePrevRaw10,returnsOpenPrevRaw10,returnsClosePrevMktres10,returnsOpenPrevMktres10,returnsOpenNextMktres10,universe
0,0,2007-02-01 22:00:00+00:00,A.N,Agilent Technologies Inc,2606900.000,32.190,32.170,0.006,0.005,nan,nan,-0.002,0.001,nan,nan,0.035,1.000
1,1,2007-02-01 22:00:00+00:00,AAI.N,AirTran Holdings Inc,2051600.000,11.120,11.080,0.005,-0.007,nan,nan,-0.079,-0.088,nan,nan,0.028,0.000
2,2,2007-02-01 22:00:00+00:00,AAP.N,Advance Auto Parts Inc,1164800.000,37.510,37.990,-0.012,0.026,nan,nan,0.014,0.045,nan,nan,0.024,1.000
3,3,2007-02-01 22:00:00+00:00,AAPL.O,Apple Inc,23747329.000,84.740,86.230,-0.012,0.016,nan,nan,-0.049,-0.037,nan,nan,-0.007,1.000
4,4,2007-02-01 22:00:00+00:00,ABB.N,ABB Ltd,1208600.000,18.020,18.010,0.012,0.025,nan,nan,0.013,0.020,nan,nan,-0.018,1.000
5,5,2007-02-01 22:00:00+00:00,ABC.N,AmerisourceBergen Corp,1657300.000,52.370,52.400,-0.000,0.008,nan,nan,0.089,0.078,nan,nan,0.059,1.000
6,6,2007-02-01 22:00:00+00:00,ABD.N,ACCO Brands Corp,1186200.000,23.630,24.130,-0.021,-0.007,nan,nan,0.005,0.027,nan,nan,-0.044,0.000


In [4]:
market_data.isnull().sum()

Unnamed: 0                      0
time                            0
assetCode                       0
assetName                       0
volume                          0
close                           0
open                            0
returnsClosePrevRaw1            0
returnsOpenPrevRaw1             0
returnsClosePrevMktres1     15980
returnsOpenPrevMktres1      15988
returnsClosePrevRaw10           0
returnsOpenPrevRaw10            0
returnsClosePrevMktres10    93010
returnsOpenPrevMktres10     93054
returnsOpenNextMktres10         0
universe                        0
dtype: int64

In [5]:
market_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4072956 entries, 0 to 4072955
Data columns (total 17 columns):
Unnamed: 0                  int64
time                        object
assetCode                   object
assetName                   object
volume                      float64
close                       float64
open                        float64
returnsClosePrevRaw1        float64
returnsOpenPrevRaw1         float64
returnsClosePrevMktres1     float64
returnsOpenPrevMktres1      float64
returnsClosePrevRaw10       float64
returnsOpenPrevRaw10        float64
returnsClosePrevMktres10    float64
returnsOpenPrevMktres10     float64
returnsOpenNextMktres10     float64
universe                    float64
dtypes: float64(13), int64(1), object(3)
memory usage: 528.3+ MB


In [6]:
for columnname in market_data.columns:
    if market_data[columnname].dtype=='float64':
        market_data[columnname]=market_data[columnname].astype('float32')

market_data['time'] =  pd.to_datetime(market_data['time'], format='%Y-%m-%d %H:%M:%S+00:00')

market_data['assetName']=market_data['assetName'].astype('category')
market_data['assetCode']=market_data['assetCode'].astype('category')
market_data['universe'] = market_data['universe'].astype('bool')
market_data.time=market_data.time.dt.date

market_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4072956 entries, 0 to 4072955
Data columns (total 17 columns):
Unnamed: 0                  int64
time                        object
assetCode                   category
assetName                   category
volume                      float32
close                       float32
open                        float32
returnsClosePrevRaw1        float32
returnsOpenPrevRaw1         float32
returnsClosePrevMktres1     float32
returnsOpenPrevMktres1      float32
returnsClosePrevRaw10       float32
returnsOpenPrevRaw10        float32
returnsClosePrevMktres10    float32
returnsOpenPrevMktres10     float32
returnsOpenNextMktres10     float32
universe                    bool
dtypes: bool(1), category(2), float32(12), int64(1), object(1)
memory usage: 268.4+ MB


In [7]:
print('Memory occupied before downcasting:',initial.sum())
print('Memory occupied after downcasting :',market_data.memory_usage().sum())
print('Total memory saved                :',round(market_data.memory_usage().sum()*100/initial.sum(), 2),'%')

NameError: name 'initial' is not defined

In [ ]:
market_data=pd.read_csv('/Users/bluegrey/Downloads/market_train_from.csv')
print(market_data.shape)
# (4072956, 17)

market_data.head(7)

In [ ]:
market_data=pd.read_csv('/Users/bluegrey/Downloads/market_train_full.csv')
print(market_data.shape)
# (4072956, 17)

market_data.head(7)